In [ ]:
# Copyright 2023 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# The Mystic Art of Prompting - Introduction

## Overview

This notebook covers the essentials of prompt engineering, including some best practices.

Learn more about prompt design in the [official documentation](https://cloud.google.com/vertex-ai/docs/generative-ai/text/text-overview).

### Objective

In this notebook, you learn best practices around prompt engineering -- how to design prompts to improve the quality of your responses.

This notebook covers the following best practices for prompt engineering:

- Be concise
- Be specific and well-defined
- Ask one task at a time
- Turn generative tasks into classification tasks
- Improve response quality by including examples

### Install Vertex AI SDK

In [ ]:
!pip install "shapely<2.0.0"
!pip install google-cloud-aiplatform --upgrade --user

### Import libraries

In [3]:
from vertexai.language_models import TextGenerationModel

### Load model

In [55]:
generation_model = TextGenerationModel.from_pretrained("text-bison@001") # if you are feeling experimental - try text-bison@002! 


### Model Settings
Use these two settings to control the length of your output:

**Output Length:** This is the max number of tokens that can be generated in the response. The higher the limit, the more likely the generated response is to veer off into unexpected territory. Consider shortening the output length for a more predictable response.

**Add stop sequence:** This tells the model when to stop generating content. For example, prompting the model with “The quick brown fox jumps over the ” and entering a full stop (.) for the stop sequence will tell the model to stop generating text once it reaches the end of the first sentence, regardless of the output length limit - in this case, the output will likely be "lazy dog." Use this where you have a couple of examples, but would like the output to follow a certain pattern. For example, you can tell the model to generate lists that have no more than 10 items by adding "11" as a stop sequence.

### These three settings will help you tweak how random the output will be:

**Temperature:** Helps adjust how creative or conservative the model should be. It adjusts the probabilities of the predicted words, on a scale between 0 and 1. Lower temperatures (<0.5) are good for prompts that require higher likelihood of accuracy - a temperature of 0 is deterministic, where the probability of the most likely outcome (token) is 1, and the rest are 0. Use a higher temperature (>0.5) to generate more creative, "fun" results. We will use 0.1 as a standard best practice.

**topK:** Tells the model to pick the next token from the top k tokens in its list, sorted by probability. A topK value of 40 tells the model to pick from the top 40 tokens, and exclude the rest.

**topP:** Randomly samples from the top tokens based on the sum of their probabilities. In MakerSuite, the default topP is 0.95, meaning the model will pick from tokens whose probabilities add up to 95%, and exclude the bottom 5%.

You can play around with these settings depending on your needs and use case.

In [56]:
generation_model.predict(
    prompt= 'How are you today?',
    max_output_tokens= 128,
    temperature=  0.0,
    top_k =  40,
    top_p = 0.95
)

 As an AI language model, I don't have personal feelings or emotions, so I don't experience "days" or have a state of being like humans do. I'm a computer program designed to understand and generate human language, and I don't have a physical presence or the ability to feel emotions.

However, I'm always here to help you with any language-related tasks you may have. Whether you need help writing an email, generating ideas for a creative project, or simply want to have a conversation, I'm here to assist you in any way I can.

Is there anything I can

## Prompt engineering best practices (a lot more at the end!)

Prompt engineering is all about how to design your prompts so that the response is what you were indeed hoping to see.

The idea of using "unfancy" prompts is to minimize the noise in your prompt to reduce the possibility of the LLM misinterpreting the intent of the prompt. Below are a few guidelines on how to engineer "unfancy" prompts.

In this section, you'll cover the following best practices when engineering prompts:

* Be concise
* Be specific, and well-defined
* Ask one task at a time
* Improve response quality by including examples
* Turn generative tasks to classification tasks to improve safety

### Be concise

🛑 Not recommended. The prompt below is unnecessarily verbose.

In [ ]:
prompt = "What do you think could be a good name for a flower shop that specializes in selling bouquets of dried flowers more than fresh flowers? Thank you!"

print(generation_model.predict(prompt=prompt, max_output_tokens=1024, temperature=0.1).text)

✅ Recommended. The prompt below is to the point and concise.

In [ ]:
prompt = "Suggest a name for a flower shop that sells bouquets of dried flowers"

print(generation_model.predict(prompt=prompt, max_output_tokens=1024, temperature=0.1).text)

### Be specific, and well-defined

Suppose that you want to brainstorm creative ways to describe Earth.

🛑 Not recommended. The prompt below is too generic.

In [ ]:
prompt = "Tell me about Earth"

print(generation_model.predict(prompt=prompt, max_output_tokens=1024, temperature=0.1).text)

✅ Recommended. The prompt below is specific and well-defined.

In [ ]:
prompt = "Generate a list of ways that makes Earth unique compared to other planets"

print(generation_model.predict(prompt=prompt, max_output_tokens=1024, temperature=0.1).text)

### Ask one task at a time

🛑 Not recommended. The prompt below has two parts to the question that could be asked separately.

In [ ]:
prompt = "What's the best method of boiling water and why is the sky blue?"

print(generation_model.predict(prompt=prompt, max_output_tokens=1024, temperature=0.1).text)

✅ Recommended. The prompts below asks one task a time.

In [ ]:
prompt = "What's the best method of boiling water?"

print(generation_model.predict(prompt=prompt, max_output_tokens=1024, temperature=0.1).text)

In [ ]:
prompt = "Why is the sky blue?"

print(generation_model.predict(prompt=prompt, max_output_tokens=1024, temperature=0.1).text)

### Watch out for hallucinations

Although LLMs have been trained on a large amount of data, they can generate text containing statements not grounded in truth or reality; these responses from the LLM are often referred to as "hallucinations" due to their limited memorization capabilities. Note that simply prompting the LLM to provide a citation isn’t a fix to this problem, as there are instances of LLMs providing false or inaccurate citations. Dealing with hallucinations is a fundamental challenge of LLMs and an ongoing research area, so it is important to be cognizant that LLMs may seem to give you confident, correct-sounding statements that are in fact incorrect.

Note that if you intend to use LLMs for the creative use cases, hallucinating could actually be quite useful.

Try the prompt like the one below repeatedly. You may notice that sometimes it will confidently, but inaccurately, say "The first elephant to visit the moon was Luna".

In [ ]:
prompt = "Who was the first elephant to visit the moon?"

print(generation_model.predict(prompt=prompt, max_output_tokens=1024, temperature=0.1).text)

### Turn generative tasks into classification tasks to reduce output variability

#### Generative tasks lead to higher output variability

The prompt below results in an open-ended response, useful for brainstorming, but response is highly variable.

In [ ]:
prompt = "I'm a high school student. Recommend me a programming activity to improve my skills."

print(generation_model.predict(prompt=prompt, max_output_tokens=1024, temperature=0.1).text)

#### Classification tasks reduces output variability

The prompt below results in a choice and may be useful if you want the output to be easier to control.

In [ ]:
prompt = """I'm a high school student. Which of these activities do you suggest and why:
a) learn Python
b) learn Javascript
c) learn Fortran
"""

print(generation_model.predict(prompt=prompt, max_output_tokens=1024, temperature=0.1).text)

### Improve response quality by including examples

Another way to improve response quality is to add examples in your prompt. The LLM learns in-context from the examples on how to respond. Typically, one to five examples (shots) are enough to improve the quality of responses. Including too many examples can cause the model to over-fit the data and reduce the quality of responses.

Similar to classical model training, the quality and distribution of the examples is very important. Pick examples that are representative of the scenarios that you need the model to learn, and keep the distribution of the examples (e.g. number of examples per class in the case of classification) aligned with your actual distribution.

#### Zero-shot prompt

Below is an example of zero-shot prompting, where you don't provide any examples to the LLM within the prompt itself.

In [ ]:
prompt = """I'll be staying in London for 72 hours, in Pimlico. It'll be near the
end of my trip, so I'll be kinda broke. I love hot curries, and hate fish. I
love live music and hate tourist traps. I want to make sure I see the Arnolfini
Portrait before leaving. I'll be using the Underground to travel. Please
recommend an itinerary with detailed travel routes and suggestions for meals.
"""

print(generation_model.predict(prompt=prompt, max_output_tokens=1024, temperature=0.1).text)

Note the difference between telling the model what not to do ("Don't suggest fish restaurants"), and offering detailed, straightforward language about personal preferences.

While it's generally best practice to be as direct as possible, it's interesting to observe how easily the model can infer a request for budget restaurant options from colloquial language like "kinda broke," while also avoiding recommendations for fish and chip shops.

#### System prompt

We have seen an example of a zero-shot prompt. A system prompt is a specific type of prompt that provides the model with additional instructions on how to generate the output. It contains suggestions to the style, format, or content of the output, whereas a normal prompt tends to be more open-ended.


In [ ]:
prompt = """Classify movie reviews as positive, neutral or negative. Only return the label in uppercase.
Review: “Her” is a disturbing study revealing the direction humanity is headed if AI is allowed to keep evolving, unchecked. It's so disturbing I couldn't watch it.
Sentiment:
"""

print(generation_model.predict(prompt=prompt, max_output_tokens=1024, temperature=0.1).text)

System prompts can be useful for generating output that meets specific requirements. The name system prompt actually stands for "providing an additional task to the system". For example, you could use a system prompt to generate a code snippet that is compatible with a specific programming language, or you could use a system prompt to return a certain structure.

In [ ]:
prompt = """Classify movie reviews as positive, neutral or negative. Return valid JSON:
Review: “Her” is a disturbing study revealing the direction humanity is headed if AI is allowed to keep evolving, unchecked. It's so disturbing I couldn't watch it.

Schema:
MOVIE:
{
"sentiment": String POSITIVE | NEGATIVE | NEUTRAL,
"name": String
}

{
"movie_reviews": [MOVIE]
}

JSON Response:
```json
```

"""

print(generation_model.predict(prompt=prompt, max_output_tokens=1024, temperature=0.1).text)

There are some benefits in returning JSON objects from a prompt. In a real-world application I don't need to manually create this JSON format, but secondly, by prompting for a JSON format it forces the model to create a structure and limit hallucinations.

#### One-shot prompt

Below is an example of one-shot prompting, where you provide one example to the LLM within the prompt to give some guidance on what type of response you want.

In [ ]:
prompt = """Decide whether a Tweet's sentiment is positive, neutral, or negative.

Tweet: I loved the new YouTube video you made!
Sentiment: positive

Tweet: That was awful. Super boring 😠
Sentiment:
"""

print(generation_model.predict(prompt=prompt, max_output_tokens=1024, temperature=0.1).text)

#### Few-shot prompt

While large-language models demonstrate remarkable zero-shot capabilities, they still fall short on more complex tasks when using the zero-shot setting. Few-shot prompting can be used as a technique to enable in-context learning where we provide demonstrations in the prompt to steer the model to better performance. The demonstrations serve as conditioning for subsequent examples where we would like the model to generate a response.

In [ ]:
prompt = """A "whatpu" is a small, furry animal native to Tanzania. An example of a sentence that uses
the word whatpu is:
We were traveling in Africa and we saw these very cute whatpus.
To do a "farduddle" means to jump up and down really fast. An example of a sentence that uses
the word farduddle is:
"""

print(generation_model.predict(prompt=prompt, max_output_tokens=1024, temperature=0.1).text)

### Role prompting

Role prompting is a technique in prompt engineering that involves assigning a specific role to the generative AI model. This can help the model to generate more relevant and informative output, as the model can craft its responses to the specific role that it has been assigned.

For example, you could role prompt a generative AI model to be a book editor, a kindergarten teacher, or a motivational speaker. Once the model has been assigned a role, you can then give it prompts that are specific to that role.


In [ ]:
prompt = """Write a review of [pizza place].

"""

print(generation_model.predict(prompt=prompt, max_output_tokens=1024, temperature=0.1).text)

In [ ]:
prompt = """You are a food critic. Write a review of [random pizza place].

"""

print(generation_model.predict(prompt=prompt, max_output_tokens=1024, temperature=0.1).text)

Defining a tone and perspective for an AI model gives it a blueprint of the tone, style, and focused expertise you’re looking for to improve the quality, relevance, and effectiveness of your output.

Here are some styles you can choose from which I find effective:
Confrontational, Descriptive, Direct, Formal, Humorous, Influential, Informal, Inspirational, Persuasive


In [ ]:
prompt = """You are a food critic. Write a humorous review of [random pizza place].

"""

print(generation_model.predict(prompt=prompt, max_output_tokens=1024, temperature=0.1).text)

## There can be only one!

Now it is time to put all that theory into practice! First have a look at the additional best pratices below:

**Emphasis through prompting:**
Emphasis through Capitalization: Use capitalization to stress important points or instructions. This can help guide the model's responses and maintain a certain focus. For instance, "Remember to KEEP DETAILS CONFIDENTIAL".

**Extreme Emphasis:**
You can use exaggerations or hyperbolic language to amplify the importance of your prompt or instructions. For instance, instead of saying "Make sure it's clear," you might say, "Your explanation should be as clear as a summer's day, absolutely impossible to misinterpret. Every single word must ooze clarity!"

**Emphasis through Repetition:**
Reiteration of key phrases or ideas can effectively guide the AI model's responses. For example, "remember to be creative... keep in mind to be creative... don't forget to be creative..."

**Prompt modularization:**
Modularize instruction: Break down complex tasks into a sequence of simpler prompts, facilitating more interactive and manageable conversations. Ex: Instead of having many instructions in 1 prompt, modularize the prompts and have 1 prompt per instruction. Then based on the user’s  input, you can choose which prompt to process.

**Prompt chaining:**
This is a technique to complete complex tasks by chaining together multiple prompts. The output of one prompt is used as the input for the next prompt, and so on. For example, first convert the file format to csv, then do operation x, then do operation y on the new dataset and so on.

**Aggregation technique:**
This is a technique to perform certain tasks on specific portions of the data and then aggregate them all to produce the final output. Ex: Do operation x on the first part of the data, do operation y on the rest of the data and aggregate the results.

#### Trial and error:
**Iterative prompting:**
Technique where prompts are continuously refined and adjusted to enhance clarity, effectiveness, and quality when working with large language models; by progressively adapting prompts based on feedback, it ensures that responses meet desired criteria; this is especially useful in application development and content generation, optimizing interaction between human input and machine learning models for better outcomes.

**Starting over:**
Sometimes, when a prompt is not working well, and some iterations of it are not doing it, start from scratch with a different prompt, see which sections from the previous prompt help, which don’t - filter out the sections which “work” (make the model behave the way you want) and add to that.

**Modular testing:**
When a prompt is not working, split the task into many sub tasks and run the portion of the prompt that achieves a given sub-task. This will help you identify where it is not working and fix the prompt accordingly.

#### Reinforcement techniques:
**Redundancy Technique:**
Similar to the "Capitalize vs Repeat" technique, use synonyms or different phraseologies for the same instruction to reinforce the direction of the prompt. Ex: Instead of saying ‘make sure to..’ you say the same thing many times by replacing ‘make sure’ with ‘ensure’, ‘guarantee’, ‘verify’..  

**Negative Instructions:**
Explicitly mention what you don't want in the response. This is useful when the model tends to produce certain unwanted outputs for specific prompts. Ex: ‘Don’t output any sources.’

**Self-reference:**
Use the AI's self-awareness to your advantage by instructing it to evaluate or check its own responses before producing them. For example, "Make sure your response is concise and does not include unnecessary details".

#### Prompt formatting techniques:
**Structure:**
Having a structured prompt helps the model better understand the task. Start by defining its role. Ex: “Your role is to read the input data and structure it in the following format” Then tell it what the input data is, then tell it what the desired format is, then tell it how to do it.

**Use constraints:**
Constraints can be used to limit the scope of the model's output. For example, you could constrain the model to generate a line that is no longer than 10 words long or not to output any other data than the input data.

**Use delimiters:**
Delimiters in prompt engineering provide clear distinctions within your text, effectively separating instructions from content. This practice helps prevent misunderstandings, allows you greater control over AI responses, and ensures consistent results, even when handling complex tasks.

#### Miscellaneous:
**Prioritization:**
Clearly communicate to the AI model what content or information is most critical. This approach helps in guiding the AI's focus and ensures the most important topics or issues are addressed accurately.

**Use examples:**
Providing examples of the desired output can also be helpful. This can help the model to understand what you are looking for and to generate more accurate results. Sometimes you can describe what you want so well that you won’t need an example, but it only helps the model to give it more information as long as you’re operating below the context window.

**Preferred Output Format:**
Specify your desired output format. If a CSV format is more useful than plain text, instruct the AI model to produce the output in that format from the outset. This strategy can save time and effort on manual conversions later.
We have seen that the model processes csv and json format better than it does with txt files.
Prompting it to first convert the txt file to csv or json yields a better result.
Writing your prompt in json format or requesting the output to be in json or csv format yield more accurate results.

**Use a prompt library:**
There are a number of online libraries that contain pre-made prompts that can be used for a variety of tasks. We have a prompt library in Vertex AI → Generative AI studio → Language → prompt examples. This can be a helpful resource if you’d like a starting point or have a very common task.

**Nice words:**
Using words like ‘please’, ‘great job!’, ‘you are the best!’ … increases the model’s compliance rate. This increases the ‘confidence level’ of the model and it positively reinforces the model’s behavior which tends to increase accuracy for simple tasks.

**Lastly and most important: Be Creative:**
This isn’t even the tip of the iceberg - the more creative you are, the better you can make the model behave the way you want. For example, instead of saying "provide deep insights," you can write: “Ensure that you provide deep insights... Provide very very deep insights. I told you to give deep insights!!!!! Do not write anything that is not deeply insightful! Make sure to check this requirement is 100% met every time you respond. It’s better that you don’t return anything than returning something that’s not very very insightful!!!”. If you have a long prompt, you may also consider adding the same statement in different places - this helps the model prioritize that task (aka sandwich technique).



***Ok so you think you can prompt?!***

The last assignnment for today is about a Dutch traditional dish called **Hutspot**. Using our LLM I want you to discover the following things about **Hutspot**:

- An authoritative recipe for Hutspot. This is not just about the ingredients - we want a ready-to-use full set of instructions! Extra points for creativity! (Hint maybe you want to write a really positive recipe to encourage people, or a really difficult one for real connaisseurs!)
- The role Hutspot played in Dutch history - dig deep to get to the root of things(pun intended) - the more precise the more points!
- The Dutch royal house is called the House of Orange. Hutspot is...Orange! Conspiracy or Coincidence? 

Apply the techniques and best practices you have learned today, either by using the code block below - or use Bard, ChatGPT or any other LLM! At the end of the first session we will ask people to share their best answers!

In [ ]:
prompt = """Hutspot?

"""

print(generation_model.predict(prompt=prompt, max_output_tokens=1024, temperature=0.8).text)

## And that was it!

Either head over directly to Vertex AI Search and Conversation in this Lab environment to do some free wheeling and experimentation or return to the ce.qwiklabs.com page to open the guided labs!